In [1]:
# zaimportowane biblioteki

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import random
import re
import os

import unicodedata

from io import open

In [3]:
# niestety, wszystko co jest zwiazane z ML/DL musi byc robione pod gpu :(
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
# montujemy dysk googla na własne potrzeby

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
# https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html
# korpus dialogów z filmów

!curl -O http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 9684k  100 9684k    0     0  15.9M      0 --:--:-- --:--:-- --:--:-- 15.9M


In [8]:
# rozpakowujemy i czyścimy niepotrzebne pliki

!unzip cornell_movie_dialogs_corpus.zip
!mv 'cornell movie-dialogs corpus/' 'data/'
!rm -rf '/content/__MACOSX'

Archive:  cornell_movie_dialogs_corpus.zip
   creating: cornell movie-dialogs corpus/
  inflating: cornell movie-dialogs corpus/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/cornell movie-dialogs corpus/
  inflating: __MACOSX/cornell movie-dialogs corpus/._.DS_Store  
  inflating: cornell movie-dialogs corpus/chameleons.pdf  
  inflating: __MACOSX/cornell movie-dialogs corpus/._chameleons.pdf  
  inflating: cornell movie-dialogs corpus/movie_characters_metadata.txt  
  inflating: cornell movie-dialogs corpus/movie_conversations.txt  
  inflating: cornell movie-dialogs corpus/movie_lines.txt  
  inflating: cornell movie-dialogs corpus/movie_titles_metadata.txt  
  inflating: cornell movie-dialogs corpus/raw_script_urls.txt  
  inflating: cornell movie-dialogs corpus/README.txt  
  inflating: __MACOSX/cornell movie-dialogs corpus/._README.txt  


In [11]:
!head -10 data/movie_characters_metadata.txt

u0 +++$+++ BIANCA +++$+++ m0 +++$+++ 10 things i hate about you +++$+++ f +++$+++ 4
u1 +++$+++ BRUCE +++$+++ m0 +++$+++ 10 things i hate about you +++$+++ ? +++$+++ ?
u2 +++$+++ CAMERON +++$+++ m0 +++$+++ 10 things i hate about you +++$+++ m +++$+++ 3
u3 +++$+++ CHASTITY +++$+++ m0 +++$+++ 10 things i hate about you +++$+++ ? +++$+++ ?
u4 +++$+++ JOEY +++$+++ m0 +++$+++ 10 things i hate about you +++$+++ m +++$+++ 6
u5 +++$+++ KAT +++$+++ m0 +++$+++ 10 things i hate about you +++$+++ f +++$+++ 2
u6 +++$+++ MANDELLA +++$+++ m0 +++$+++ 10 things i hate about you +++$+++ f +++$+++ 7
u7 +++$+++ MICHAEL +++$+++ m0 +++$+++ 10 things i hate about you +++$+++ m +++$+++ 5
u8 +++$+++ MISS PERKY +++$+++ m0 +++$+++ 10 things i hate about you +++$+++ ? +++$+++ ?
u9 +++$+++ PATRICK +++$+++ m0 +++$+++ 10 things i hate about you +++$+++ m +++$+++ 1


In [12]:
!head -10 data/movie_conversations.txt

u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L271', 'L272', 'L273', 'L274', 'L275']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L276', 'L277']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L280', 'L281']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L363', 'L364']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L365', 'L366']


In [13]:
!head -10 data/movie_lines.txt

L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!
L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!
L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.
L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?
L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.
L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow
L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.
L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No
L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I'm kidding.  You know how sometimes you just become this "persona"?  And you don't know how to quit?
L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?


In [14]:
def load_lines(file_name):
  lines = {}
  
  with open(file_name, 'r', encoding='iso-8859-1') as f:
    for line in f:
      values = line.split(' +++$+++ ')
      line = {}
      
      for i, field in enumerate(['line_id', 'characted_id', 'movie_id', 'character', 'text']):
        line[field] = values[i]
        
      lines[line['line_id']] = line

  return lines

In [15]:
lines = load_lines('data/movie_lines.txt')
list(lines.items())[:5]

[('L1045',
  {'characted_id': 'u0',
   'character': 'BIANCA',
   'line_id': 'L1045',
   'movie_id': 'm0',
   'text': 'They do not!\n'}),
 ('L1044',
  {'characted_id': 'u2',
   'character': 'CAMERON',
   'line_id': 'L1044',
   'movie_id': 'm0',
   'text': 'They do to!\n'}),
 ('L985',
  {'characted_id': 'u0',
   'character': 'BIANCA',
   'line_id': 'L985',
   'movie_id': 'm0',
   'text': 'I hope so.\n'}),
 ('L984',
  {'characted_id': 'u2',
   'character': 'CAMERON',
   'line_id': 'L984',
   'movie_id': 'm0',
   'text': 'She okay?\n'}),
 ('L925',
  {'characted_id': 'u0',
   'character': 'BIANCA',
   'line_id': 'L925',
   'movie_id': 'm0',
   'text': "Let's go.\n"})]

In [16]:
def load_conversations(file_name, lines):
  conversations = []
  
  with open(file_name, 'r', encoding='iso-8859-1') as f:
    for line in f:
      values = line.split(' +++$+++ ')

      coversation = {}
      for i, field in enumerate(['character_1_id', 'character_2_id', 'movie_id', 'lines']):
        coversation[field] = values[i]

      line_ids = re.compile('L[0-9]+').findall(coversation['lines'])
      coversation['lines'] = [lines[id]['text'] for id in line_ids]

      conversations.append(coversation)
      
  return conversations

In [17]:
conversations = load_conversations('data/movie_conversations.txt', lines)
conversations[:5]

[{'character_1_id': 'u0',
  'character_2_id': 'u2',
  'lines': ['Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\n',
   "Well, I thought we'd start with pronunciation, if that's okay with you.\n",
   'Not the hacking and gagging and spitting part.  Please.\n',
   "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?\n"],
  'movie_id': 'm0'},
 {'character_1_id': 'u0',
  'character_2_id': 'u2',
  'lines': ["You're asking me out.  That's so cute. What's your name again?\n",
   'Forget it.\n'],
  'movie_id': 'm0'},
 {'character_1_id': 'u0',
  'character_2_id': 'u2',
  'lines': ["No, no, it's my fault -- we didn't have a proper introduction ---\n",
   'Cameron.\n',
   "The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\n",
   'Seems like she could get a date easy enough...\n'],
  'movie_id': 'm0'},
 {'character_

In [18]:
# https://stackoverflow.com/a/518232/2809427
# usuwanie diakrytyków

def to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

In [19]:
def normalize(s):
    s = to_ascii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

In [20]:
def extract_pairs(conversations):
  pairs = []
  
  for conversation in conversations:
    for i in range(len(conversation['lines']) - 1):
      q_line = normalize(conversation['lines'][i].strip())
      a_line = normalize(conversation['lines'][i + 1].strip())
      
      if q_line and a_line:
        pairs.append([q_line, a_line])

  return pairs

In [21]:
pairs = extract_pairs(conversations)
pairs[:5]

[['can we make this quick ? roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad . again .',
  'well i thought we d start with pronunciation if that s okay with you .'],
 ['well i thought we d start with pronunciation if that s okay with you .',
  'not the hacking and gagging and spitting part . please .'],
 ['not the hacking and gagging and spitting part . please .',
  'okay . . . then how bout we try out some french cuisine . saturday ? night ?'],
 ['you re asking me out . that s so cute . what s your name again ?',
  'forget it .'],
 ['no no it s my fault we didn t have a proper introduction', 'cameron .']]

In [26]:
# musimy ograniczyć nieco nasz zbiór, dlatego interesują nas dialogi, które mają w sobvie nie więcej niż MAX_LENGTH słów.

MAX_LENGTH = 10

def filter_pair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH

def filter_pairs(pairs):
    return [pair for pair in pairs if filter_pair(pair)]

In [27]:
# i tworzymy pary dialogowe

pairs = filter_pairs(pairs)
pairs[:5]

[['there .', 'where ?'],
 ['you have my word . as a gentleman', 'you re sweet .'],
 ['hi .', 'looks like things worked out tonight huh ?'],
 ['you know chastity ?', 'i believe we share an art instructor'],
 ['have fun tonight ?', 'tons']]

In [29]:
# tokeny dla słowa ktore nie znajduje się w słowniku (o tym troche niżej) PAD
# dla rozpoczęcia wypowiedzi SOS
# dla zakończenia wypowiedzi EOS

PAD_token = 0
SOS_token = 1
EOS_token = 2

In [45]:
# tworzymy słownik (wszystkich słow występujących w naszych odifltrowanych dialogach)
# w środku znajduje się licznik słów (ile razy dane słowo wystąpiło w korpusie)

class Vocabulary:
    def __init__(self):
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3

    def add_sentence(self, sentence):
        for word in sentence.split(' '):
            self.add_word(word)

    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    def trim(self, min_count):
        keep_words = []
        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3

        for word in keep_words:
            self.add_word(word)

    def get_index(self, word):
        if word not in self.word2index:
            return PAD_token
        else:
            return self.word2index[word]

In [46]:
# question vocabulary vs answer vocabulary
# slownik pytan i odpowiedzi

qvoc = Vocabulary()
avoc = Vocabulary()

In [47]:
# zobaczmy jak wyglądają słowniki (ile mają słów po załadowaniu)
for pair in pairs:
  qvoc.add_sentence(pair[0])
  avoc.add_sentence(pair[1])

print(qvoc.num_words)
print(avoc.num_words)

14297
13898


In [48]:
qvoc.trim(3)
avoc.trim(3)

print(qvoc.num_words)
print(avoc.num_words)

4930
4742


In [49]:
def sentence_to_tensor(voc, sentence):
   indexes = [voc.get_index(word) for word in sentence.split(' ')]
   indexes.append(EOS_token)

   return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def pair_to_tensor(qvoc, avoc, pair):
   input = sentence_to_tensor(qvoc, pair[0])
   output = sentence_to_tensor(avoc, pair[1])

   return (input, output)

In [50]:
pair = random.choice(pairs)
(input, output) = pair_to_tensor(qvoc, avoc, pair)

print(input)
print(output)

tensor([[ 48],
        [  0],
        [ 40],
        [194],
        [ 79],
        [  5],
        [400],
        [ 14],
        [  2]], device='cuda:0')
tensor([[  18],
        [ 174],
        [ 524],
        [2656],
        [ 848],
        [  18],
        [   6],
        [ 132],
        [   8],
        [   2]], device='cuda:0')


In [51]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [52]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden, encoder_outputs):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden, None

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [116]:
def step(input, target, encoder, decoder, enopt, deopt, criterion, max_length = MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    enopt.zero_grad()
    deopt.zero_grad()

    input_length = input.size(0)
    target_length = target.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden

    for di in range(target_length):
        decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
        loss += criterion(decoder_output, target[di])
        decoder_input = target[di]

    loss.backward()

    _ = nn.utils.clip_grad_norm_(encoder.parameters(), 50)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), 50)

    enopt.step()
    deopt.step()

    return loss.item() / target_length

In [117]:
def train(name, encoder, decoder, qvoc, avoc, pairs, iterations = 100000, milestone = 5000):

  # optymalizator
  encoder_optimizer = optim.Adam(encoder.parameters(), lr=0.0001)
  decoder_optimizer = optim.Adam(decoder.parameters(), lr=0.0001)
  
  # negative log likelihood loss
  loss_fn = nn.NLLLoss()
  avg_loss = 0

  # losujemy pary treningowe
  training_pairs = [pair_to_tensor(qvoc, avoc, random.choice(pairs)) for i in range(iterations)]
  
  for i in range(1, iterations):
    input = training_pairs[i][0]
    target = training_pairs[i][1]
    
    # dla kazdej pary treningowej szacujemy blad i sumujemy
    loss = step(input, target, encoder, decoder, encoder_optimizer, decoder_optimizer, loss_fn)
    avg_loss += loss
    
    # uczymy korzystając z średniej na 5000 par (milestone)
    if i % milestone == 0:
      avg_loss = avg_loss / milestone
      print('%d %.4f' % (i, avg_loss))
      avg_loss = 0

      torch.save(encoder.state_dict(), '/content/' + name + '_encoder_it-' + str(i) + '.model')
      torch.save(decoder.state_dict(), '/content/' + name + '_decoder_it-' + str(i) + '.model')

In [129]:
embed_size = 512
hidden_size = 1024
num_layers = 1

# tworzymy dwa modele
encoder = EncoderRNN(qvoc.num_words, hidden_size).to(device)
decoder = DecoderRNN(hidden_size, avoc.num_words).to(device)

In [120]:
train('seq2seq_chatbot', encoder, decoder, qvoc, avoc, pairs)

In [100]:
# zapisanie wytrenowanych modeli (encodera i decodera)

torch.save(encoder, '/content/seq2seq_chatbot_encoder.model')
torch.save(decoder, '/content/seq2seq_chatbot_decoder.model')

In [139]:
!curl -O https://raw.githubusercontent.com/dotjabber/alk-nlp-intro/main/seq2seq_chatbot_encoder.model

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 43.2M  100 43.2M    0     0  46.3M      0 --:--:-- --:--:-- --:--:-- 46.2M


In [140]:
# załadowanie wytrenowanych modeli
# można je pobrać stąd:
# encoder: https://raw.githubusercontent.com/dotjabber/alk-nlp-intro/main/seq2seq_chatbot_encoder.model
# decoder: https://raw.githubusercontent.com/dotjabber/alk-nlp-intro/main/seq2seq_chatbot_decoder.model

# !curl -O https://raw.githubusercontent.com/dotjabber/alk-nlp-intro/main/seq2seq_chatbot_encoder.model
# !curl -O https://raw.githubusercontent.com/dotjabber/alk-nlp-intro/main/seq2seq_chatbot_decoder.model

encoder = torch.load('/content/seq2seq_chatbot_encoder.model')
decoder = torch.load('/content/seq2seq_chatbot_decoder.model')

In [141]:
def evaluate(encoder, decoder, qvoc, avoc, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = sentence_to_tensor(qvoc, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)
        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(avoc.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [143]:
evaluate(encoder, decoder, qvoc, avoc, 'hello')

['hello', '.', '<EOS>']